This assignment is part II of your evaluation process. You should address every question of the assignment on the cell below. If you decide to write any explanation, you should do so commenting on your code cells or by creating a new markdown cell.

This is an **optional** assignment and a great opportunity to show you more advanced skills.

**Name:** Wandercleyson Scheidegger

**CPF**:

**Date**: 15/09/2021

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings("ignore")

# Assignment 2 - Hypothesis Testing  (optional )


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', \
    'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', \
        'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', \
            'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': \
                'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', \
                    'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', \
                        'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', \
                            'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', \
                                'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', \
                                    'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', \
                                        'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri',\
                                             'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina',\
                                                  'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska',\
                                                       'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', \
                                                           'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', \
                                                               'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', \
                                                                   'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [140]:
def get_list_of_university_towns():
    
    #carrega o dataset
    df = pd.read_csv("university_towns.txt", sep='/n', header=None).rename(columns={0:'State'})

    #limpa os dados
    df.loc[df['State'].str.find("[edit]") == -1, 'RegionName'] = df['State']
    df.loc[df['State']==df['RegionName'], 'State'] = np.nan
    df['State'] = df['State'].str.replace(r'\[.*','').fillna(method='ffill')
    df['RegionName'] = df['RegionName'].str.replace(r'\W*\(.*','')

    
    df.dropna(subset=['RegionName'], inplace=True)
    
    return df


In [141]:
a = get_list_of_university_towns()
a

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [142]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    #carrega o dataset e renomeia colunas
    df = pd.read_excel('gdplev.xls', header=7).\
        drop(columns=['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'])\
            .rename(columns={'Unnamed: 4':'YearQ', 'Unnamed: 6':'GDP'})

    #filtro requerido
    df = df[df['YearQ']>'2000q1']

    #faz o shif dos dados
    inicio_rec = df[(df['GDP'] > df['GDP'].shift(-1)) & (df['GDP'].shift(-1) > df['GDP'].shift(-2))]

    return inicio_rec.iloc[0,0]

In [143]:
b = get_recession_start()
b

'2008q2'

In [144]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    df = pd.read_excel('gdplev.xls', header=7).\
        drop(columns=['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'])\
            .rename(columns={'Unnamed: 4':'YearQ', 'Unnamed: 6':'GDP'})

    df = df[df['YearQ']>'2000q1']

    inicio_rec = df[(df['GDP'] > df['GDP'].shift(-1)) & (df['GDP'].shift(-1) > df['GDP'].shift(-2))].iloc[0,0]

    df = df[df['YearQ'] > inicio_rec]

    
    fim_rec = df[(df['GDP'] > df['GDP'].shift(1)) & (df['GDP'].shift(1) > df['GDP'].shift(2))].copy()
    
    return fim_rec.iloc[0,0]

In [145]:
c = get_recession_end()
c

'2009q4'

In [146]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    df = pd.read_excel('gdplev.xls', header=7).\
        drop(columns=['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'])\
            .rename(columns={'Unnamed: 4':'YearQ', 'Unnamed: 6':'GDP'})

    df = df[df['YearQ']>'2000q1']

    inicio_rec = df[(df['GDP'] > df['GDP'].shift(-1)) & (df['GDP'].shift(-1) > df['GDP'].shift(-2))].iloc[0,0]

    df = df[df['YearQ'] > inicio_rec]

    inicio_rec = get_recession_start()
    fim_rec = get_recession_end()

    df = df[(df['YearQ'] >= inicio_rec) & (df['YearQ'] <= fim_rec)]
    
    inib = df[df['GDP']==df['GDP'].min()]

    return inib.iloc[0,0]

In [147]:
d = get_recession_bottom()
d

'2009q2'

In [148]:
def convert_housing_data_to_quarters():
    #dic pra facilitar o replace
    dicionario = {'01':'q1','02':'q1','03':'q1','04':'q2','05':'q2','06':'q2','07':'q3','08':'q3','09':'q3','10':'q4','11':'q4','12':'q4'}

    df = pd.read_csv('City_Zhvi_AllHomes.csv')

    df['State'] = df['State'].map(states)

    df.drop(columns=['SizeRank', 'Metro', 'CountyName'], inplace=True)


    df_ = df.set_index(['RegionID','State','RegionName']).stack().\
        reset_index().rename(columns={'level_3':'Ano-M',0:'PrecoMedio'})

    #map melhor que apply (muitas amostras)
    df_['YearQtr'] = df_['Ano-M'].str[0:4] + (df_['Ano-M'].str[-2:].map(dicionario))

    df__ = df_[df_['YearQtr']>='2000q1'].copy().drop(columns=['Ano-M'])

    
    return df__.groupby(['RegionID','State','RegionName','YearQtr']).mean().unstack()


In [149]:
e = convert_housing_data_to_quarters()
e

PrecoMedio                 \
YearQtr                                         2000q1         2000q2   
RegionID State        RegionName                                        
3304     New Jersey   Absecon            113500.000000  117633.333333   
3305     Maryland     Accokeek                     NaN            NaN   
3310     Oklahoma     Ada                          NaN            NaN   
3312     Georgia      Adairsville         85966.666667   87300.000000   
3315     Tennessee    Adams              109833.333333  111100.000000   
...                                                ...            ...   
399305   Pennsylvania Swatara Township   109166.666667  110166.666667   
737786   California   The Sea Ranch      394600.000000  410500.000000   
737787   Arizona      San Tan Valley      99333.333333  105333.333333   
737790   California   Eastvale           258766.666667  267233.333333   
737791   Connecticut  Town of Mansfield  146333.333333  150733.333333   

                                                                       \
YearQtr                                         2000q3         2000q4   
RegionID State        RegionName                                        
3304     New Jersey   Absecon            118033.333333  118833.333333   
3305     Maryland     Accokeek                     NaN            NaN   
3310     Oklahoma     Ada                          NaN            NaN   
3312     Georgia      Adairsville         85100.000000   86033.333333   
3315     Tennessee    Adams              112133.333333  113100.000000   
...                                                ...            ...   
399305   Pennsylvania Swatara Township   109766.666667  109100.000000   
737786   California   The Sea Ranch      431066.666667  448633.333333   
737787   Arizona      San Tan Valley     110066.666667  116166.666667   
737790   California   Eastvale           266933.333333  261900.000000   
737791   Connecticut  Town of Mansfield  151100.000000  153166.666667   

                                                                       \
YearQtr                                         2001q1         2001q2   
RegionID State        RegionName                                        
3304     New Jersey   Absecon            121033.333333  121700.000000   
3305     Maryland     Accokeek                     NaN            NaN   
3310     Oklahoma     Ada                          NaN            NaN   
3312     Georgia      Adairsville         86466.666667   86200.000000   
3315     Tennessee    Adams              114500.000000  115966.666667   
...                                                ...            ...   
399305   Pennsylvania Swatara Township   109633.333333  110900.000000   
737786   California   The Sea Ranch      457666.666667  459466.666667   
737787   Arizona      San Tan Valley     117733.333333  113633.333333   
737790   California   Eastvale           268033.333333  277433.333333   
737791   Connecticut  Town of Mansfield  155533.333333  158266.666667   

                                                                       \
YearQtr                                         2001q3         2001q4   
RegionID State        RegionName                                        
3304     New Jersey   Absecon            123466.666667  125766.666667   
3305     Maryland     Accokeek                     NaN            NaN   
3310     Oklahoma     Ada                          NaN            NaN   
3312     Georgia      Adairsville         88666.666667   88733.333333   
3315     Tennessee    Adams              115433.333333  115566.666667   
...                                                ...            ...   
399305   Pennsylvania Swatara Township   111466.666667  112500.000000   
737786   California   The Sea Ranch      434066.666667  427866.666667   
737787   Arizona      San Tan Valley     111966.666667  112566.666667   
737790   California   Eastvale           283933.333333  279566.666667   
737791   Connecticut  Town

In [153]:
def run_ttest():

    #intervalo
    inicio = get_recession_start()
    fim = get_recession_bottom()
    
    #lista das universidades
    iniv = get_list_of_university_towns()
    iniv['Type'] = 'Universidade'
    
    #conversao dos dados em trimestres
    df = convert_housing_data_to_quarters().PrecoMedio[[inicio,fim]].copy()

    #proporção
    df['prop'] = df[inicio] / df[fim]

    df = df[df['prop'].notna()].copy()

    df = df.merge(iniv, on=['State','RegionName'], how='left')

    #adiciona a label faltante
    df['Type'] = df['Type'].fillna('NotUnivers')
    
    universidade = df[df['Type']=='Universidade']
    notunivers = df[df['Type']=='NotUnivers']
    
    p = ttest_ind(universidade['prop'], notunivers['prop'])[1]

    
    if universidade['prop'].mean() < notunivers['prop'].mean():
        better = "university town"
    else:
        better = "non-university town"

    if p < 0.01:
        reject = True
    else:
        reject = False
    
    return reject, p, better

In [151]:
rej, p, melhor = run_ttest()